## Day 12: Passage Pathing

With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

```
start-A
start-b
A-c
A-b
b-d
A-end
b-end
```

This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:

```
    start
    /   \
c--A-----b--d
    \   /
     end
```

Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

Given these rules, there are 10 paths through this example cave system:

```
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
```

(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:

```
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
```

The 19 paths through it are as follows:

```
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
```

Finally, this even larger example has 226 paths through it:

```
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
```

How many paths through this cave system are there that visit small caves at most once?

In [158]:
def read_input(filename):
    """
    Read and parse cave system input

    """

    # Parse cave data
    # Each line is a set of the two connected caves
    # Use sets as order is irrelevant
    with open(filename, "r") as infile:
        cave_data = [set(l.strip().split("-")) for l in infile]

    return cave_data


def find_cave_paths(filename):
    """
    Find all paths through cave system subject to rules:

    Distinct paths
    - start at start
    - end at end
    - don't visit small (lowercase) caves more than once.

    """
    
    # Key observation - the cave network is a Directed Acyclic Graph
    # We are trying to generate all valid graphs based on its edges (the input)

    cave_data = read_input(filename)


    
    def generate_paths(path=["start"], paths=[]):
        """
        Generate valid DAGs (i.e. paths through cave) based on the edges

        """
        


        # Current cave label
        current_cave = path[-1]

        # Flag tracks if path has reached a dead end or not
        path_stopped = True
        
        # Part 2 logic
        # Check if any lowercase cave has now been visited once already
        # If it has, set max_lowercase back to 0
        lowercase_caves = [x for x in path if x.islower() and x not in ['start', 'end']]
        if sum([lowercase_caves.count(x) > 1 for x in set(lowercase_caves)]) > 0:
            max_lowercase = 0
        else:
            max_lowercase = 0 # CHANGE TO 1 FOR PART 2 ⚠️

                
        # Get valid moves from the cave we're in now
        if current_cave == "start":
            moves = [x for x in cave_data if current_cave in x]
        elif current_cave == "end":
            moves = []
        else:
            # This includes the logic for checking if we have already
            # visited a 'lowercase' cave
            moves = [
                x
                for x in cave_data
                if (current_cave in x)
                and ("start" not in x)
                and (
                    (path.count(list(x.difference(set([current_cave])))[0]) <= max_lowercase)
                    or list(x.difference(set([current_cave])))[0].isupper()
                )
            ]
            


        # Iterate over all valid next moves
        for edge in moves:

            # Append next move to the path
            new_path = path + list(edge.difference(set([current_cave])))

            # Recursion 🥴
            generate_paths(new_path, paths)

            # Keep going while we're able to iterate over these moves
            path_stopped = False

        if path_stopped:
            paths.append(path)

        return paths

    paths = generate_paths()

    # All we need to do now is filter out the paths that don't end up in the 'end' cave
    final_paths = [x for x in paths if x[-1] == "end"]
    



    print("Found " + str(len(final_paths)) + " valid paths through the cave")



    
    return 0


find_cave_paths("day12-example.txt")



Found 10 valid paths through the cave


0

In [159]:
find_cave_paths("day12-input.txt")



Found 4104 valid paths through the cave


0

## Part Two

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.

Now, the 36 possible paths through the first example above are:

```    
start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
```

The slightly larger example above now has 103 paths through it, and the even larger example now has 3509 paths through it.

Given these new rules, how many paths through this cave system are there?

In [156]:
find_cave_paths("day12-input.txt")



Found 119760 valid paths through the cave


0